# Summery
虽然是十一月，但是现在好像是十一月二十三，十一月要结束了。距离上次好好学术已经过去两个月了。这两个月就干了三件事，一个是开题答辩，另外一个是修改论文，还有一个就是学会了latex。现在一想起来，就是不知道自己在焦虑什么，明明没有选择，但是却没有行动。这时候好像不需要焦虑，因为自己本就一无所有，真正需要选择的时候才是我们要考虑的时候。
     
做学术，秉承着哪里有热度就往哪钻的想法，至少还能占个坑位，准备就是说下午真的要开始了。就逮着SAM2，先过一下论文，然后开始源码看看。

## 11/24 周日  
清华：三篇关于高性能计算的论文
- 在时空维度提升扩散生成模型计算效率。空间维度改进 $DiT(Diffusion Transformer)$ ,计算对角线**atten**权重，对角线权重共享；时间维度在时间步上通过分析相似时间步的权重，然后权重共享。
- 在扩散图像生成模型探究新式量化方法，通过对不同层的动态量化使其精度影响降到最小。并探究了量化不同层对图像生成质量的不同影响。
- 蒸馏自回归模型到几个step用于图像生成。通过噪声伪权重不用计算完前面所有权重才能得到当前权重，使用噪声代替特定位置权重可以一次性生成多个位置的输出。  

**NeurIPS 2024** 计算机视觉
1. 一个通用的驾驶世界模型，通过当前状态和给出的控制信号预测未来行为。提出比**nuscences**更大更通用的数据集；基于**stable diffusion**，由生成模型到预测模型。加入最后三帧做长自回归视频预测，提升运动信息权重保证图像生成质量。   

2. 基于压缩(**compression**)的角度重新理解基于 transformer 的语义分割。通过借助降维压缩，PCA的思想，推导基于transformer的语义分割模型。省略FFN网络，保留self-attention和cross-attention，大大减少模型网络参数，借用PCA思想，保证attention内高维特征参数正交，增强模型对噪声干扰的稳健性。   

3. 利用大模型的语义直觉来提升SAM对[**伪装样本**]的分割性能。这里面用来扩散模型来生成去除主体的背景图片，使用SAM来分类，使用LLM直觉预测伪装样本的语义标签。   

4. 基于带噪的泊松融合自蒸馏深度优化。高分辨率深度估计分patch，一致性不强会有局部闪烁，低分辨率计算高效，但是边缘模糊。使用泊松融合得到最均衡性能最有效的结果。   

5. SAM 稳健性研究。SAM对高频信息的敏感，在频域破坏高频信息；SAM分割没有真正的标签，语义解耦不彻底，在空域里打散前景背景---->以此来攻击SAM。

6. [**TTA(Test-time Adaptation)**]将模型用于新数据集上推理测试时动态调整自身模型权重，提升模型性能。<扩散模型>

7. 探究雷达位置对3D场景理解的影响。

8. EMR-Merging: Tuning-Free High-Performance Model Merging 模型融合：使用多模型融合得到对多任务理解的能力。


## 11/25 周一
**Benchmark**是用于评价算法或系统性能的一套标准化的测试工具或数据集。它通常包含：
- 标准数据集：被广泛接受和使用的数据集，用于统一评估不同方法的性能。
- 评价指标：预定义的、统一的指标（如准确率、F1-score、mAP等）来量化性能。
- 任务定义：明确规定的任务或问题（如图像分类、目标检测、语义分割等）。

**Baseline**是指在特定任务或实验中，用于比较的参考方法或模型。它是一个“起点”或“最低标准”，用来衡量新方法的改进程度。特点：
1. 简单性：通常选择简单或传统的方法作为baseline。
2. 比较性：用于对比新方法的性能是否优于已有方法。
3. 多样性：可以包含多个参考方法，提供更丰富的对比信息。

///--------------------------------------------------------------------------

#### 点积（Dot Product）
点积是两个向量之间的代数操作，其公式为： 
$$ \mathbf{a} \cdot \mathbf{b}=\sum_{i=1}^{n} a_{i} b_{i} $$
$$ \mathbf{a} \cdot \mathbf{b}=\|\mathbf{a}\|\|\mathbf{b}\| \cos \theta $$

#### 余弦相似度（Cosine Similarity）
余弦相似度是一个归一化的相似性度量，定义为：
$$cosine-similarity(\mathbf{a}, \mathbf{b})= \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\|\|\mathbf{b}\| \cos \theta}$$ 

## 11/26 周二
- 再看一遍 *SAM* 论文实现细节。
- 查看 *image encoder* 源码，采用 **ViTDet**[1] 窗口注意力处理高分辨率图片，**MAE** 预训练方式。
- 查看 *prompt encoder* 源码，嵌入方法与位置编码。考虑原本数据prompt，加上一个由二维映射到高维的随机空间频率位置编码。
- 制作分享 PPT ，开始我的学术团队交流活动。

[1] Li, Yanghao, et al. "Exploring plain vision transformer backbones for object detection." European conference on computer vision(ECCV). Cham: Springer Nature Switzerland, 2022.

## 11/27 周三
*mask decoder* 基于 MaskFormer[1-2] 的方式计算掩码解码:     
- **tokens:** 由 iou_token 和 mask_tokens 以及 sparse_prompt_embeddings concat 组成一个整体的序列
- **src:** image_embeddings + dese_prompt_embeddings
- **position_encode:** 来自 dense_prompt 的 image_pe

上面三者进入 attention 中相互融合:
- *iou_token* 经过一个 mlp 得到 iou_pred 作为每个 mask 的 *iou scores* (batch, num_mask)     
- *mask_tokens* 中的每一个 token 分别经过一个 hyper_mlp 得到 *hyper_in* (batch, num_mask, dim/8)      
- *src* 经过上采样网络 up_scaled ，高宽变为原来的四倍，维度变为原来1/8，得到 *upscaled_embedding* (batch, dim/8, H, W)    
--->$masks = hyper\_in @ upscaled\_embedding$     


[1] Cheng, Bowen, et al. "Per-pixel classification is not all you need for semantic segmentation." Advances in neural information processing systems(NeurIPS) 34 (2021): 17864-17875.    
[2] Cheng, Bowen, et al. "Masked-attention mask transformer for universal image segmentation." Proceedings of the IEEE/CVF conference on computer vision and pattern recognition(CVPR). 2022.